In [31]:
import datasets
import utils
import gmm_impute
import gmm_dae
import pickle_utils as pu
import numpy as np
import pandas as pd
from sklearn.mixture import BayesianGaussianMixture

import imp
imp.reload(utils)
imp.reload(datasets)
imp.reload(gmm_impute)
imp.reload(gmm_dae)

<module 'gmm_dae' from '/mnt/disk1/MIMIC/ts-baselines/timeless-imputation/gmm_dae.py'>

In [17]:
dsets = datasets.datasets()
baseline = datasets.benchmark({'MICE': datasets.memoize(utils.impute_mice),
                               'MissForest': datasets.memoize(utils.impute_missforest)}, dsets)

/mnt/disk1/MIMIC/ts-baselines/timeless-imputation/utils.py:26: RuntimeWarning: invalid value encountered in subtract
  sq_diff = (original_df-multiple_imputed_df)**2


In [18]:
BGMMs = {}
for name, df in dsets.items():
    m = BayesianGaussianMixture(n_components=100, covariance_type='full', max_iter=10000)
    m.fit(df.values)
    BGMMs[len(df)] = gmm_impute.prune_infinite_gmm(df.values, m)

In [19]:
m = BGMMs[len(dsets["BostonHousing"])]
from gmm_impute import mask_matrix
mask = np.zeros([m['means'].shape[1]], dtype=np.bool)
mask[[1,2,5,7,9,10]] = True
print("mask:", mask)
inp = dsets["BostonHousing"].values[0]

def compute_classically():
    K_12 = mask_matrix(m['covariances'],mask,~mask)
    K_22__1 = np.linalg.inv(mask_matrix(m['covariances'],~mask,~mask))
    K_1222 = K_12 @ K_22__1
    K_21 = mask_matrix(m['covariances'],~mask,mask)
    K_11 = mask_matrix(m['covariances'],mask,mask)

    diff = np.expand_dims(inp[~mask] - m['means'][:,~mask], axis=2)
    return (m['means'][:,mask] + np.squeeze(K_1222 @ diff, axis=2)), (K_11 - K_1222 @ K_21)

def compute_newly():
    K12 = m['covariances']*(mask[:,None])*(~mask)

mask: [False  True  True False False  True False  True False  True  True False]


In [35]:
%%time
def gmm_impute_f(_path, dataset):
    _dataset = dataset.values
    ndarray = gmm_impute._gmm_impute(BGMMs[len(dataset)], _dataset)
    return list(pd.DataFrame(a, index=dataset.index, columns=dataset.columns) for a in ndarray)
benchmark_gmm = datasets.benchmark({'GMM': gmm_impute_f}, dsets)

CPU times: user 1min 4s, sys: 2min 31s, total: 3min 36s
Wall time: 27.9 s


In [36]:
%%time
def ff(p, d):
    return gmm_dae.gmm_dae_impute(p, d, BGMMs[len(d)])
gmm_dae_df = datasets.benchmark({'GMM_dae': ff}, dsets)

CPU times: user 1h 50min 23s, sys: 8min 51s, total: 1h 59min 15s
Wall time: 22min 9s


In [37]:
print("RECALL: the GMMs are learned from the full data.")
pd.concat([baseline, benchmark_gmm, gmm_dae_df], axis=1)

RECALL: the GMMs are learned from the full data.


MICE            MissForest         GMM  \
                                    RMSE  RMSE_best  RMSE_best        RMSE   
BostonHousing MCAR_rows  0.1   90.569383  31.579920  24.778823  259.776112   
                         0.3  111.491826  38.751257  32.529281  365.874376   
                         0.5  132.949103  44.513093  39.598019  446.386916   
                         0.7  135.844986  47.167001  46.250560  561.618699   
                         0.9  165.095831  58.274468  53.095003  701.442756   
              MCAR_total 0.1   93.918937  31.220683  19.712973  193.999699   
                         0.3   96.601112  32.757876  29.476191  288.240238   
                         0.5  109.387762  39.315867  31.999789  342.374332   
                         0.7  130.874209  46.023247  45.229790  452.594995   
                         0.9         NaN        NaN  65.061966  639.016980   
Ionosphere    MCAR_rows  0.1    1.248583   0.469796   0.389532    0.899032   
                         0.3    1.298070   0.477468   0.410098    1.320561   
                         0.5    1.356755   0.493529   0.432965    1.712566   
                         0.7    1.438012   0.514666   0.463627    3.358118   
                         0.9    1.867148   0.618045   0.553832    5.845880   
              MCAR_total 0.1    1.207885   0.457670   0.376508    0.652394   
                         0.3    1.160458   0.435457   0.361475    0.817640   
                         0.5    1.238051   0.459572   0.392154    0.982749   
                         0.7    1.358311   0.477307   0.427948    1.632185   
                         0.9         NaN        NaN   0.578642    4.452051   

                                            GMM_dae             
                              RMSE_best        RMSE  RMSE_best  
BostonHousing MCAR_rows  0.1  21.905663  266.750682  21.864283  
                         0.3  26.108675  359.928943  25.339364  
                         0.5  33.559436  445.517413  33.570004  
                         0.7  41.480506  560.865687  41.515276  
                         0.9  51.534486  701.641007  51.713207  
              MCAR_total 0.1  15.026299  193.181227  15.016779  
                         0.3  23.180362  293.763540  23.439403  
                         0.5  26.399618  343.309882  26.384788  
                         0.7  34.009667  455.968099  34.003743  
                         0.9  47.069459  636.826734  46.672813  
Ionosphere    MCAR_rows  0.1   0.064812    0.899310   0.064593  
                         0.3   0.100307    1.325849   0.101359  
                         0.5   0.127584    1.700216   0.126783  
                         0.7   0.269795    3.358927   0.270289  
                         0.9   0.476755    5.839926   0.476502  
              MCAR_total 0.1   0.032833    0.651997   0.032438  
                         0.3   0.051375    0.818850   0.051643  
                         0.5   0.065193    0.978609   0.064648  
                         0.7   0.123133    1.635856   0.123446  
                         0.9   0.352975    4.453506   0.352567